In [1]:
from __future__ import division
import numpy as np
import scipy.linalg as slg
import math
import sys
from numpy import genfromtxt
import csv
from numpy import linalg as LA
from numpy.linalg import inv

In [2]:
def str2float(s):
    return float(s.replace('D','e'))

def CutLTMatOut(fname, head_line, index=0):
    cur_index = 0
    f = open(fname)
    status = None
    r = 0
    indices = []
    mat = []
    for line in f:
        if head_line in line:
            status = "Column Indices"
            continue
        if status == "Rows":
            splitted = line.split()
            if all([x.isnumeric() for x in splitted]):
                status = "Column Indices"
            else:
                try:
                    r = int(splitted[0])
                    floats = [str2float(s) for s in splitted[1:]]
                    if len(mat) < r:
                        mat.append(floats)
                    else:
                        mat[r-1] += floats
                except ValueError as e:
                    break
                continue
        if status == "Column Indices":
            status = "Rows"
            indices = [int(I) for I in line.split()]
            continue
    f.close()
    return mat

def toNPMatrix(mat_array, symmetry = 'symmetric'):
    dim = len(mat_array)
    mat = np.zeros((dim,dim))
    for i in range(dim):
        mat[i,:i+1] = mat_array[i]
    if symmetry == 'symmetric':
        diag = np.diag(np.diag(mat))
        return mat + mat.T - diag
    elif symmetry == 'anti-symmetric':
        return mat - mat.T


def ReadMatrix(fname, head_line, index=0):
    return np.array(CutLTMatOut(fname, head_line, index))

def ReadLowerTri(fname, head_line, index=0, symmetry = 'symmetric'):
    return toNPMatrix(CutLTMatOut(fname, head_line, index), symmetry)

def outMatrix(A):
    if np.iscomplexobj(A):
        print("Real part:")
        outMatrix(A.real)
        print()
        print("Imag part:")
        outMatrix(A.imag)
        return
    print("--------------------------------------------------------------------------------")
    curColStart, curColEnd = 0, 5
    while curColStart < A.shape[1]:
        curColEnd = min(curColStart + 5, A.shape[1])
        print("     "+"               ".join(list(map(str, range(curColStart + 1, curColEnd + 1)))))
        for i in range(A.shape[0]):
            print("%3d   "%(i+1,) + " ".join(list(map(lambda x : "%15.8e"%(x if abs(x) > 1e-8 else 0,) , A[i, curColStart : curColEnd]))))
        curColStart += 5
        print()

    print("--------------------------------------------------------------------------------")

def RMSD(V, W):
    """
    Calculate Root-mean-square deviation from two sets of vectors V and W.
    Parameters
    ----------
    V : array
        (N,D) matrix, where N is points and D is dimension.
    W : array
        (N,D) matrix, where N is points and D is dimension.
    Returns
    -------
    rmsd : float
        Root-mean-square deviation between the two vectors
    """
    diff = np.array(V) - np.array(W)
    N = len(V)
    return np.sqrt((diff * diff).sum() / N)
    

In [30]:
def getErrorMatrix(F,P,S,V,printlevel):
    FPS = np.dot(np.dot(F, P), S)

    e_AO = FPS - FPS.transpose()

    e_ON = np.dot(V.transpose(), np.dot(e_AO, V))

    if (printlevel >= 1):
        if (printlevel >= 2):
            print('F')
            outMatrix(F)
            print('P')
            outMatrix(P)
            print('FP')
            outMatrix(np.dot(F, P))
            print('S')   
            outMatrix(S)
        print('FPS')
        outMatrix(FPS)
        print('e_AO')
        outMatrix(e_AO)
        if (printlevel >= 2):
            print('V')
            outMatrix(V)
        print('e_ON')
        outMatrix(e_ON)
    return  e_ON

In [31]:
def getBmatrix(E_list):
    Bsize = len(E_list)
    B = np.zeros((Bsize+1,Bsize+1))
    
    B[0,0] = 0.00
    for i in range(1,Bsize+1):
        B[i,0] = -1
        B[0,i] = -1
        B[i,i] = np.trace(np.dot(E_list[i-1].T,E_list[i-1]))
        for j in range(i+1,Bsize+1):
            B[i,j] = np.trace(np.dot(E_list[i-1].T,E_list[j-1]))
            B[j,i] = B[i,j]
        
    return B

In [32]:
def newF(F_list, B):
    Fsize = len(F_list)
    NBasis = F_list[0].shape[0]
    
    F = np.zeros((NBasis,NBasis))
    for i in range(Fsize):
        ci = inv(B)[:,0][1:][i] * (-1)
        print('c' + str(i) + ': ', ci)
        F += ci * F_list[i]
    return F

In [60]:
fname_fulllog = "/Users/aodongliu/LiGroup/gaussian/manual_calculation/neo_small_basis_stw.log"
fname = "./Formaldehyde_STWDIIS.out"
prot_S_headline = 'Prot S Matrix:          0:'
prot_V_headline = 'V for IR=Prot          1'
prot_S = ReadLowerTri(fname_fulllog, prot_S_headline)
prot_V = ReadMatrix(fname_fulllog, prot_V_headline)


In [66]:
Fp1_headline = 'Fock Matrix1:'
Fp1 = ReadMatrix(fname, Fp1_headline)
Pp1_headline = 'Density Matrix1:'
Pp1 = ReadMatrix(fname, Pp1_headline)

e1_manual_ON = getErrorMatrix(Fp1,Pp1,prot_S,prot_V,0)
E_list = [e1_manual_ON]
F_list = [Fp1]

B = getBmatrix(E_list)
print('Manually Calculated B-matrix')
outMatrix(B)

newF_B1 = newF(F_list,B)

Manually Calculated B-matrix
--------------------------------------------------------------------------------
     1               2
  1    0.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00  5.74531055e-03

--------------------------------------------------------------------------------
c0:  1.0


In [67]:
Fp2_headline = 'Fock Matrix2:'
Fp2 = ReadMatrix(fname, Fp2_headline)
Pp2_headline = 'Density Matrix2:'
Pp2 = ReadMatrix(fname, Pp2_headline)

e2_manual_ON = getErrorMatrix(Fp2,Pp2,prot_S,prot_V,0)
E_list.append(e2_manual_ON)
F_list.append(Fp2)

B = getBmatrix(E_list)
print('Manually Calculated B-matrix')
outMatrix(B)

newF_B2 = newF(F_list,B)

Manually Calculated B-matrix
--------------------------------------------------------------------------------
     1               2               3
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00  5.74531055e-03  5.39736505e-03
  3   -1.00000000e+00  5.39736505e-03  5.07161404e-03

--------------------------------------------------------------------------------
c0:  -14.677109448504057
c1:  15.677109448504057


In [68]:
Fp3_headline = 'Fock Matrix3:'
Fp3 = ReadMatrix(fname, Fp3_headline)
Pp3_headline = 'Density Matrix3:'
Pp3 = ReadMatrix(fname, Pp3_headline)

e3_manual_ON = getErrorMatrix(Fp3,Pp3,prot_S,prot_V,0)
E_list.append(e3_manual_ON)
F_list.append(Fp3)

B = getBmatrix(E_list)
print('Manually Calculated B-matrix')
outMatrix(B)

newF_B3 = newF(F_list,B)

Manually Calculated B-matrix
--------------------------------------------------------------------------------
     1               2               3               4
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00  5.74531055e-03  5.39736505e-03  3.15496953e-04
  3   -1.00000000e+00  5.39736505e-03  5.07161404e-03  3.11951568e-04
  4   -1.00000000e+00  3.15496953e-04  3.11951568e-04  2.40086021e-04

--------------------------------------------------------------------------------
c0:  28.814033363178726
c1:  -30.80983421702675
c2:  2.995800853848024


In [69]:
Fp4_headline = 'Fock Matrix4:'
Fp4 = ReadMatrix(fname, Fp4_headline)
Pp4_headline = 'Density Matrix4:'
Pp4 = ReadMatrix(fname, Pp4_headline)

e4_manual_ON = getErrorMatrix(Fp4,Pp4,prot_S,prot_V,0)
E_list.append(e4_manual_ON)
F_list.append(Fp4)

B = getBmatrix(E_list)
print('Manually Calculated B-matrix')
outMatrix(B)

newF_B4 = newF(F_list,B)

Manually Calculated B-matrix
--------------------------------------------------------------------------------
     1               2               3               4               5
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00  5.74531055e-03  5.39736505e-03  3.15496953e-04  1.22133714e-04
  3   -1.00000000e+00  5.39736505e-03  5.07161404e-03  3.11951568e-04  1.28420628e-04
  4   -1.00000000e+00  3.15496953e-04  3.11951568e-04  2.40086021e-04  2.02859288e-04
  5   -1.00000000e+00  1.22133714e-04  1.28420628e-04  2.02859288e-04  1.75329246e-04

--------------------------------------------------------------------------------
c0:  -3.378764171173729
c1:  3.813069512997166
c2:  -6.400229300873277
c3:  6.96592395904984


In [70]:
Fp5_headline = 'Fock Matrix5:'
Fp5 = ReadMatrix(fname, Fp5_headline)
Pp5_headline = 'Density Matrix5:'
Pp5 = ReadMatrix(fname, Pp5_headline)

e5_manual_ON = getErrorMatrix(Fp5,Pp5,prot_S,prot_V,0)
E_list.append(e5_manual_ON)
F_list.append(Fp5)

B = getBmatrix(E_list)
print('Manually Calculated B-matrix')
outMatrix(B)

newF_B5 = newF(F_list,B)

Manually Calculated B-matrix
--------------------------------------------------------------------------------
     1               2               3               4               5
  1    0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
  2   -1.00000000e+00  5.74531055e-03  5.39736505e-03  3.15496953e-04  1.22133714e-04
  3   -1.00000000e+00  5.39736505e-03  5.07161404e-03  3.11951568e-04  1.28420628e-04
  4   -1.00000000e+00  3.15496953e-04  3.11951568e-04  2.40086021e-04  2.02859288e-04
  5   -1.00000000e+00  1.22133714e-04  1.28420628e-04  2.02859288e-04  1.75329246e-04
  6   -1.00000000e+00  9.05828884e-07  8.23394182e-07 -3.48498690e-07 -3.31549499e-07

     6
  1   -1.00000000e+00
  2    9.05828884e-07
  3    8.23394182e-07
  4   -3.48498690e-07
  5   -3.31549499e-07
  6    0.00000000e+00

--------------------------------------------------------------------------------
c0:  0.03605603398981608
c1:  -0.04059160380440345
c2:  0.05870702837942072
c3:  